## Inpainting

### Import Libraries

In [ ]:
#Import libraries for simulation
import os
import time
import sys
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.measure import compare_ssim as ssim

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
def make_kernel(a):
  """Transform a 2D array into a convolution kernel"""
  a = np.asarray(a)
  a = a.reshape(list(a.shape) + [1,1])
  return tf.constant(a, dtype=1)

In [ ]:
def heat_conv(input, kernel):
  """A simplified 2D convolution operation for Heat Equation"""
  input = tf.expand_dims(tf.expand_dims(input, 0), -1)

  result = tf.nn.depthwise_conv2d(input=input, filter=kernel,
                                    strides=[1, 1, 1, 1],
                                    padding='SAME')

  return result[0, :, :, 0]

In [ ]:
def show_viz(i,original, masked, mask, inpainted):
    """Show Image using matplotlib"""
    plt.figure(i)
    plt.subplot(221), plt.imshow(original, 'gray')
    plt.title('original image')
    plt.subplot(222), plt.imshow(masked, 'gray')
    plt.title('source image')
    plt.subplot(223), plt.imshow(mask, 'gray')
    plt.title('mask image')
    plt.subplot(224), plt.imshow(inpainted, 'gray')
    plt.title('inpaint result')

    plt.tight_layout()
    plt.draw()

In [ ]:
def show_ssim(original, masked, inpainted):
    """Show SSIM Difference"""
    print("SSIM : ")
    print("  Original vs. Original  : ", ssim(original,original))
    print("  Original vs. Masked    : ", ssim(original,masked))
    print("  Original vs. Inpainted : ", ssim(original,inpainted))

In [ ]:
def inpaint(masked, mask):
    # Init variable
    N = 2000
    ROOT_DIR = os.getcwd()

    # Create variables for simulation state
    U = tf.Variable(masked)
    print("Created tf variable 'U'")
    G = tf.Variable(masked)
    print("Created tf variable 'G'")
    M = tf.Variable(np.multiply(mask,1))
    print("Created tf variable 'M'")
    
    print("Constructing kernel....")
    K = make_kernel([[0.0, 1.0, 0.0],
                     [1.0, -4., 1.0],
                     [0.0, 1.0, 0.0]])
    print("Created kernel")

    dt = tf.compat.v1.placeholder(tf.float16, shape=())

    """Discretized PDE update rules"""
    """u[i,j] = u[i,j] + dt * (u[i+1,j] + u[i-1,j] + u[i,j+1] + u[i,j-1] - 4*u[i,j]) - dt * lambda_m[i,j]*(u[i,j]-g[i,j])"""

    #Tensorflow while_loop function, iterate the PDE N times.
    index_summation = (tf.constant(1), U, M, G, K)
    def condition(i, U, M, G, K):
        return tf.less(i, 100)

    def body(i,U,M,G,K):
        U_ = U + 0.1 * heat_conv(U,K) - 0.1 * M * (U-G)
        print(i)
        return tf.add(i, 1), U_, M, G, K

    #Tensorflow Session
    with tf.compat.v1.Session():
        # Initialize state to initial conditions
        tf.compat.v1.global_variables_initializer().run()

        #Run PDE using tensorflow while_loop
        t = time.time()
        uf=tf.while_loop(cond=condition, body=body, loop_vars=index_summation)[1]
        U = uf.eval()

    print("Execution Time : {} s".format(time.time()-t))

    return U

## Inpainting

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
!ls '/gdrive/My Drive/payload'

Despeckling.gdoc       image1_mask.png	image1_.png
image1_inpaint_tf.jpg  image1_ori.png


In [ ]:
# IMG_DIR = os.path.join('/gdrive', 'dataset')
DATA_DIR = '/gdrive/My Drive/SAR'

IMG_DIR = os.path.join(DATA_DIR, 'denoised')
MASK_DIR = os.path.join(DATA_DIR, 'mask')

tf.compat.v1.disable_eager_execution()

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [ ]:
original = cv2.imread(os.path.join(IMG_DIR, 'image{}_ori.png'.format(1)),0)
masked = cv2.imread(os.path.join(IMG_DIR, 'image{}_.png'.format(1)),0)
mask = cv2.imread(os.path.join(IMG_DIR, 'image{}_mask.png'.format(1)),0)

In [ ]:
original = cv2.normalize(original, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
masked = cv2.normalize(masked, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
mask = 1-cv2.normalize(mask, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

In [ ]:
print("\nFile : image{}_.png {}".format(1,masked.shape))
inpainted = inpaint(masked,mask)


File : image1_.png (13973, 13463)
Created tf variable 'U'
Created tf variable 'G'
Created tf variable 'M'
Constructing kernel....
Created kernel
Tensor("Placeholder:0", shape=(), dtype=int32)
Execution Time : 22.448574542999268 s


In [ ]:
cv2.imwrite("/gdrive/My Drive/payload/image{}_inpaint_tf.jpg".format(1),inpainted*255)

True

In [ ]:
show_ssim(original,masked,inpainted)

SSIM : 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  after removing the cwd from sys.path.


## Tensor Cores

In [ ]:
# On TPUs, use 'mixed_bfloat16' instead
from tensorflow.keras.mixed_precision import experimental as mixed_precision

policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
mixed_precision.set_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 2060, compute capability 7.5


In [ ]:
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Compute dtype: float16
Variable dtype: float32


In [ ]:
original = cv2.imread(os.path.join(IMG_DIR, 'image{}_ori.png'.format(1)),0)
masked = cv2.imread(os.path.join(IMG_DIR, 'image{}_.png'.format(1)),0)

NameError: ignored

In [ ]:
original = cv2.normalize(original, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
masked = cv2.normalize(masked, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

In [ ]:
print("  Original vs. Masked    : ", ssim(original,masked))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  """Entry point for launching an IPython kernel.
